# Mounter

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Extractor

In [0]:
import pandas as pd
import numpy as np
import json
import csv
import bz2
import os
import sys

In [0]:
def extractor(fle,writer,up_l,down_l,IP_det):
  total=[]
  source_file=bz2.BZ2File(fle,'r')
  prev='ip'
  temp_l=[]
  for i,line in enumerate(source_file):
    if i<down_l: 
      continue
    if i==up_l: 
      break
    print(i)
    data=json.loads(line)
    if 'from' not in data.keys() or 'dst_addr' not in data.keys():
      continue
    src_addr=data['from']
    dest_addr=data["dst_addr"]
    if IP_det.get(dest_addr) is None:
      continue
    temp=data['result']
    for iter in temp:
      if "result" not in iter.keys():
        continue
      data_temp=iter['result']
      for ext in data_temp:
        if "from" in ext.keys():
          if prev==str(ext["from"]): continue
          temp_l.append(ext["from"])
          prev=str(ext["from"])
    if len(temp_l)==0:
      continue
    writer.write(str(i)+","+str(src_addr)+","+str(dest_addr)+"\n")
    total.append(temp_l)  
    temp_l=[]
  down_l=up_l
  up_l=up_l+20000
  if i<down_l-1:
    flag=1
  else:
    flag=0

  return total,down_l,up_l,flag  

In [0]:
def merge(name_w,total):
  df=pd.read_csv(name_w)
  df['Hops']=total
  df.to_csv(name_w,index=False)

In [0]:
def caller_ext():
  Path="drive/My Drive/iitkgp_summer_project"
  fldr="traceroute"
  print(Path+'/'+fldr)
  files=os.listdir(Path+"/"+fldr)
  up_l=20000
  down_l=0
  i=files[0]
  fle=Path+"/"+fldr+"/"+i
  print(fle)
  flag=0
  file_l='15169_details.json'
  print(Path+'/'+file_l)
  f=open(Path+'/'+file_l,"r")
  IP_det=json.loads(f.read())
  name=Path+'/'+i[-14:-4]+'Google'
  print(name)
  if os.path.exists(name):
   
    print("error")
    exit(0)
  else:
    os.mkdir(name)

  counter_n=1
  while(flag==0):
    
    name_w=name+'/IP_rec_Google'+i[-14:-4]+"_"+str(counter_n)+".csv"
    print(name_w)
    text=open(name_w,"w")

    text.write("index,source,dest\n")
    text.close()
    writer=open(name_w,'a')

    total,down_l,up_l,flag=extractor(fle,writer,up_l,down_l,IP_det)
    writer.close()
    merge(name_w, total)  
    counter_n+=1

In [0]:
caller_ext()

 # All IP Listing

In [0]:
import pandas as pd
import json
import ast
import os

In [0]:
def preprocess(fle,path,fldr1,fldr2):
  Path="drive/My Drive/iitkgp_summer_project/Google_data"
  df=pd.read_csv(Path+'/'+fldr1+'/'+fldr2+'/'+fle)
  z=0
  df_l=df.values.tolist()
  data={}
  while z<len(df):
    lst=ast.literal_eval(df_l[z][3])
    k=df_l[z][1]
    lst.insert(0,k)
    lst.append(df_l[z][2])
    j=0
    while j<len(lst)-1:
      if data.get(lst[j])==None:
        data[lst[j]]=[lst[j+1]]
      else:
        ips=data.get(lst[j])
        ips.append(lst[j+1])
        data[lst[j]]=ips
        #print(ips)
      j+=1
    
    if data.get(lst[j])==None:
      data[lst[j]]=[]
    z+=1
  dump=path+"/"+fle[:-4]+'.json'
  print(dump)
  if os.path.exists(dump):
    os.remove(dump)
  with open(dump, 'w') as fp:
    json.dump(data, fp)

In [0]:
def preprocess_caller():
  Path="drive/My Drive/iitkgp_summer_project/Google_data"
  Path2=Path+'_json'
  os.mkdir(Path2)
  
  fldr1=os.listdir(Path)
  for i in fldr1:
    p=Path2+'/'+i
    os.mkdir(p)
    fldr2=os.listdir(Path+'/'+i)
    for j in fldr2:
      q=p+'/'+j
      os.mkdir(q)
      files=os.listdir(Path+'/'+i+'/'+j)
      for k in files:
        preprocess(k,q,i,j)

      
  
      

In [0]:
preprocess_caller()

# Destination Specific Distribution

In [0]:
import pandas as pd
import json
import os
import ast

In [0]:
def Destination_wise(details,fle,fldr1,fldr2,IP):
  Path="drive/My Drive/iitkgp_summer_project/Google_data"
  df=pd.read_csv(Path+'/'+fldr1+'/'+fldr2+'/'+fle)
  
  z=0
  while z<len(df):
    if df.iloc[z]['dest']==IP:
      if details.get(df.iloc[z]['source']) is None:
        temp=df.iloc[z]['source']
        details[temp]={}
        details[temp]['1']={}
        details[temp]['1']['Hops']=lst=ast.literal_eval(df.iloc[z]['Hops'])
        details[temp]['1']['date']=fldr1
        details[temp]['1']['time']=fldr2[5:9]
      else:
        temp=df.iloc[z]['source']
        temp_2=details.get(df.iloc[z]['source'])
        l=len(temp_2)
        details[temp][str(l+1)]={}
        num=str(l+1)
        details[temp][num]['Hops']=lst=ast.literal_eval(df.iloc[z]['Hops'])
        details[temp][num]['date']=fldr1
        details[temp][num]['time']=fldr2[5:9]
    z+=1  

  return details


In [0]:
def Destwise_caller():
  Path="drive/My Drive/iitkgp_summer_project"
  Path_2="drive/My Drive/iitkgp_summer_project/Google_data"
  detail={}
  IP='8.8.8.8'
  dump=Path+'/'+IP+"_details.json"
  
  fldr1=os.listdir(Path_2)
  for i in fldr1:
    
    fldr2=os.listdir(Path_2+'/'+i)
    for j in fldr2:
     
      files=os.listdir(Path_2+'/'+i+'/'+j)
      for k in files:
        print(Path_2+'/'+i+'/'+j+'/'+k)
        Destination_wise(detail,k,i,j,IP)

  with open(dump, 'w') as fp:
    json.dump(detail, fp)
  
      

In [0]:
Destwise_caller()

# Detail Extractor

In [0]:
import os
import csv
import pandas as pd
import json

In [0]:
def create_csv(fle,writer_file):
  f=open(fle,"r")
  data=json.loads(f.read())
  writer=open(writer_file,'w')
  writer.write("Source,Time,date\n")
  writer.close()
  writer=open(writer_file,'a')
  hops=[]
  for key in data.keys():
    temp=data.get(key)
    for i in temp:
      writer.write(key+','+data[key][i]['time']+','+data[key][i]['date']+'\n')
      hops.append(data[key][i]['Hops'])
  writer.close()
  return hops


In [0]:
def merge(name_w,total):
  df=pd.read_csv(name_w)
  df['Hops']=total
  df.to_csv(name_w,index=False)

In [0]:
Path="drive/My Drive/iitkgp_summer_project"
fle=Path+'/8.8.8.8_details.json'
file_w=fle[:-5]+'.csv'
total=create_csv(fle,file_w)
merge(file_w,total)